# COVID-19: Understanding the impact of Lockdown on Growth Factor,Recovery Rate, Mortality Rate, and Contagion rate of the pathogen(𝛽) and  Recovery Rate(𝛾) in India Using SIR Model

> ## CONTENTS

[1. Introduction](#section-one)

[2. Importing Necessary Modules and Reading Data](#section-two)

[2.1 Reading Global Data](#section-three)

[2.2 Exploratory Data Analysis](#section-four)

[2.3 Situation of India](#section-five)

[2.4 Testing in India](#section-six)

[2.5 State Wise Analysis](#section-seven)

[3. Forecasting](#section-eight)

[3.1 Prophet Model](#section-nine)

[3.2 ARIMA Model](#section-ten)

[4. Lockdown Analysis in India](#section-eleven)

[4.1 Growth Factor](#section-twelve)

[4.2 SIR Model](#section-thirteen)

[4.3 Analysing Data Before Lockdown (Phase 0)](#section-fourteen)

[4.4 Analysis of Lockdown 1.0 (25th March - 14th April)](#section-fifteen)

[4.5 Analysis of Lockdown 2.0](#section-sixteen)

[4.6 Analysis of Lockdown 3.0](#section-seventeen)



<a id="section-one"></a>
> ## 1. Introduction
>  COVID-19 is spearding at a rapid pace worldwide, most inviduals infected with COVID-19 are asymptotic or show mild symptoms, to contain the spread of COVID-19 and keep infection at a manageable level, many countries have institued lockdowns and social distancing including in India.
> In India, lockdown was announced in Phases - 
* 1st Phase - 25th March - 14 April
* 2nd Phase - 15th April - 3 May
* 3rd Phase -  4 May - Present

> Additionally, interventions such as social distacing and quarantine of infectected individual.
> > In this notebook, I have analyzed the cumulative growth rate, mortality rate, testing rate, also contagion rate and recovery rate using SIR Model in each phases of lockdown.

### *I will keep updating this notebook continuosly with new visualization on updated data.*

#### Data for India has been taken from http://covid19india.org/, CSSEGISandData

#### Global data is taken from Kaggle


<a id="section-two"></a>
> ## 2. Importing Necessary Modules and Reading Data

In [ ]:
import pandas as pd
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import minimize
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import OrdinalEncoder
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima_model import ARIMA
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import datetime

<a id="section-three"></a>
> ## 2.1 Reading Global Data

In [ ]:
ncov_df = pd.read_csv('../input/novel-corona-virus-2019-dataset/covid_19_data.csv')

ncov_df['ObservationDate'] = pd.to_datetime(ncov_df['ObservationDate']) 

ncov_df["Country"] = ncov_df["Country/Region"].replace(
    {
        "Mainland China": "China",
        "Hong Kong SAR": "Hong Kong",
        "Taipei and environs": "Taiwan",
        "Iran (Islamic Republic of)": "Iran",
        "Republic of Korea": "South Korea",
        "Republic of Ireland": "Ireland",
        "Macao SAR": "Macau",
        "Russian Federation": "Russia",
        "Republic of Moldova": "Moldova",
        "Taiwan*": "Taiwan",
        "Cruise Ship": "Others",
        "United Kingdom": "UK",
        "Viet Nam": "Vietnam",
        "Czechia": "Czech Republic",
        "St. Martin": "Saint Martin",
        "Cote d'Ivoire": "Ivory Coast",
        "('St. Martin',)": "Saint Martin",
        "Congo (Kinshasa)": "Congo",
    }
)
ncov_df["Province"] = ncov_df["Province/State"].fillna("-").replace(
    {
        "Cruise Ship": "Diamond Princess cruise ship",
        "Diamond Princess": "Diamond Princess cruise ship"
    }
)

# More data from CSSEGISandData
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')


<a id="section-four"></a>
> ## 2.2 Exploratory Data Analysis

In [ ]:
world_confirmed = confirmed_df[confirmed_df.columns[-1:]].sum()
world_recovered = recovered_df[recovered_df.columns[-1:]].sum()
world_deaths = deaths_df[deaths_df.columns[-1:]].sum()
world_active = world_confirmed - (world_recovered - world_deaths)

labels = ['Active','Recovered','Deceased']
sizes = [world_active,world_recovered,world_deaths]
color= ['#66b3ff','green','red']
explode = []

print("No.of Confirmed Cases", world_confirmed)
print("No. of Recovered Cases",world_recovered)
print("No. of Deaths",world_deaths)
print("No. of Active Cases",world_active)

for i in labels:
    explode.append(0.01)
    
plt.figure(figsize= (10,10))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=9, explode =explode,colors = color)
centre_circle = plt.Circle((0,0),0.70,fc='white')

fig = plt.gcf()
fig.gca().add_artist(centre_circle)
plt.title('World COVID-19 Cases',fontsize = 10)
plt.axis('equal')  
plt.tight_layout()

> ## Observation
Till 17th May, There were total 4713620 confirmed cases, 1733963 recovered cases, 315185 Deaths,and 3294842 Active Cases.

<a id="section-five"></a>
> ## 2.3 Situation of India

> ### Reading Data for India

In [ ]:
## India Complete State Wise Data
df1 = pd.read_csv('https://api.covid19india.org/csv/latest/raw_data1.csv',parse_dates=['Date Announced'])
df2 = pd.read_csv('https://api.covid19india.org/csv/latest/raw_data2.csv',parse_dates=['Date Announced'])
df3 = pd.read_csv('https://api.covid19india.org/csv/latest/raw_data3.csv',parse_dates=['Date Announced'])
df1.append(df2)
df_ind = df1.append(df3)
df_ind.head(2)

In [ ]:
# Time Series Over all Data
ts_india = pd.read_csv('../input/updated-case-time-series-data/case_time_series 17.csv')
ts_india.head(2)


In [ ]:
ts_india['Date'] =  pd.to_datetime(ts_india['Date'], format='%d/%m/%Y')

In [ ]:
ts_confirmed = ts_india.drop(columns=['Daily Confirmed','Daily Recovered','Total Recovered','Daily Deceased','Total Deceased'])

ts_confirmed.head(3)

In [ ]:
tc_param = ts_india.drop(columns=['Daily Confirmed','Daily Recovered','Daily Deceased'])
tc_param.head()

In [ ]:
## Adding Active Cases Features
tc_param['Total Active Cases'] = tc_param['Total Confirmed'] - tc_param['Total Recovered'] - \
                                     tc_param['Total Deceased']

In [ ]:
# color pallette
cnf = '#393e46' # confirmed - grey
dth = '#ff2e63' # death - red
rec = '#21bf73' # recovered - cyan
act = '#fe9801' # active case - yellow
temp = tc_param[tc_param['Date'] == max(ts_india['Date'])]

# subset columns
temp = temp[['Date', 'Total Active Cases', 'Total Deceased', 'Total Recovered']]

# rename columns
temp.columns = ['Date', 'Active', 'Deaths', 'Cured']

# melt into longer format
tm = temp.melt(id_vars="Date", value_vars=['Active', 'Deaths', 'Cured'])

# plot
fig_1 = px.treemap(tm, path=["variable"], values="value", height=250, width=800,
                 color_discrete_sequence=[act, rec, dth], title='Latest stats')
fig_1.data[0].textinfo = 'label+text+value'
fig_1.show()

In [ ]:
# stacked bar chart

# subset columns
temp = tc_param[['Date', 'Total Active Cases', 'Total Deceased', 'Total Recovered']]

# rename columns
temp.columns = ['Date', 'Active', 'Deaths', 'Cured']

# melt data
temp = temp.melt(id_vars="Date", value_vars=['Cured', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')
temp.head()

# plot
fig_2 = px.bar(temp, x="Date", y="Count", color='Case', height=540, 
               title='Cases over time', color_discrete_sequence = [rec, dth, act])
fig_2.show()

In [ ]:
from plotly.subplots import make_subplots

fig_c = px.bar(tc_param, x="Date", y="Total Confirmed", color_discrete_sequence=['#000000'])
fig_d = px.bar(tc_param, x="Date", y="Total Deceased", color_discrete_sequence=['#ff677d'])
fig_r = px.bar(tc_param, x="Date", y="Total Recovered", color_discrete_sequence=['#649d66'])

fig_dc = px.bar(ts_india, x="Date", y="Daily Confirmed", color_discrete_sequence=['#323232'])
fig_dd = px.bar(ts_india, x="Date", y="Daily Deceased", color_discrete_sequence=['#cd6684'])
fig_dr = px.bar(ts_india, x="Date", y="Daily Recovered", color_discrete_sequence=['#16817a'])

fig = make_subplots(rows=3, cols=2, shared_xaxes=False, 
                    horizontal_spacing=0.14, vertical_spacing=0.1,
                    subplot_titles=('Confirmed cases', 'New cases', 
                                    'Deaths reported', 'New deaths', 
                                    'Cured', 'New cured'))

fig.add_trace(fig_c['data'][0], row=1, col=1)
fig.add_trace(fig_dc['data'][0], row=1, col=2)

fig.add_trace(fig_d['data'][0], row=2, col=1)
fig.add_trace(fig_dd['data'][0], row=2, col=2)

fig.add_trace(fig_r['data'][0], row=3, col=1)
fig.add_trace(fig_dr['data'][0], row=3, col=2)

fig.update_layout(height=1400)

> ## Observations

>  In India, till 17th May there are almost 95K+ confirmed cases, with 5079 new cases on 17th itself out of which 36k+ have been recovered, above 3K have deceased.
>  Seeing at curves daily new cases in india is growing at a very fast rate.

<a id="section-six"></a>
> ## 2.4 Testing in India

In [ ]:
df_w_db = pd.read_csv('../input/covid19testing/tested_worldwide.csv')
df_w_db.head(2)

In [ ]:
df_ind = df_w_db[df_w_db['Country_Region']=='India']
df_ind.head(2)

In [ ]:
fig_ind = go.Figure()
fig_ind.add_trace(go.Scatter(x=df_ind['Date'], y=df_ind['total_tested'], name='India', \
                        ))

fig_ind.layout.update(title_text='Total no. of tests done in India',xaxis_showgrid=False, width=800, xaxis_title='Date',\
                yaxis_showgrid=False, yaxis_title='No. of Tests Done',
        height=600,font=dict(
#         family="Courier New, monospace",
        size=14,
        color="black"
    ))
fig_ind.layout.plot_bgcolor = 'white'
fig_ind.layout.paper_bgcolor = 'white'
fig_ind.show()

> ## Observation 


> In India till now 2.2M+ tests have been done.

<a id="section-seven"></a>
> ## 2.5 State Wise Analysis

In [ ]:
#Reading state wise data
state_wise = pd.read_csv('https://api.covid19india.org/csv/latest/state_wise.csv')

In [ ]:
state_d=state_wise.drop(columns=['State_code','Last_Updated_Time','Delta_Confirmed','Delta_Recovered','Delta_Deaths','State_Notes'])


In [ ]:
state_db=state_d[state_d.State!='Total']

In [ ]:
# sort based on confirmed cases
latest = state_db.sort_values('Confirmed', ascending=False).reset_index(drop=True)

# background color
latest.style\
    .background_gradient(cmap="Blues", subset=['Confirmed', 'Active'])\
    .background_gradient(cmap="Greens", subset=['Recovered'])\
    .background_gradient(cmap="Reds", subset=['Deaths'])\

In [ ]:
fig = px.bar(state_db.sort_values('Confirmed', ascending=False).sort_values('Confirmed', ascending=True),
             x="Confirmed", y="State", 
             title='Total Confirmed Cases', 
             text='Confirmed', 
             orientation='h', 
             width=800, height=800, range_x = [0, max(state_db['Confirmed'])])
fig.update_traces(marker_color='#670404', opacity=0.8, textposition='inside')

fig.update_layout(plot_bgcolor='white')
fig.show()

> ## Observation - 
In India Maharashtra is the most Affected State, followed by Gujrat, Tamil Nadu and Delhi.

<a id="section-eight"></a>
>  ## 3. Forecasting 

<a id="section-nine"></a>
> ## 3.1 Prophet Model
 
Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality, plus holiday effects. It works best with time series that have strong seasonal effects and several seasons of historical data. Prophet is robust to missing data and shifts in the trend, and typically handles outliers well.

Prophet is open source software released by Facebook’s Core Data Science team. [https://facebook.github.io/prophet/]



In [ ]:
df1 = confirmed_df.groupby('Country/Region').sum().reset_index()
dates = list(confirmed_df.columns[4:])
dates = list(pd.to_datetime(dates))
dates_india = dates[8:]

In [ ]:
k = df1[df1['Country/Region']=='India'].loc[:,'1/22/20':]
india_confirmed = k.values.tolist()[0] 
data = pd.DataFrame(columns = ['ds','y'])
data['ds'] = dates
data['y'] = india_confirmed

prop=Prophet()
prop.fit(data)
future=prop.make_future_dataframe(periods=30)
prop_forecast=prop.predict(future)
forecast = prop_forecast[['ds','yhat']].tail(30)
fig = plot_plotly(prop, prop_forecast)
fig = prop.plot(prop_forecast,xlabel='Date',ylabel='Confirmed Cases')

> ## Observation: 
Using Prophet Model on India Data, it is predicting over 175000 cases in the next 15 days

<a id="section-ten"></a>
> ## 3.2 ARIMA Model
>  ARIMA Model is also a forecasting model, provided by -
[https://www.statsmodels.org/stable/generated/statsmodels.tsa.arima_model.ARIMA.html]

In [ ]:
arima = ARIMA(data['y'], order=(5, 1, 0))
arima = arima.fit(trend='c', full_output=True, disp=True)
forecast = arima.forecast(steps= 30)
pred = list(forecast[0])

start_date = data['ds'].max()
prediction_dates = []
for i in range(30):
    date = start_date + datetime.timedelta(days=1)
    prediction_dates.append(date)
    start_date = date
plt.figure(figsize= (15,10))
plt.xlabel("Dates",fontsize = 20)
plt.ylabel('Total cases',fontsize = 20)
plt.title("Predicted Values for the next 15 Days" , fontsize = 20)
plt.plot_date(y= pred,x= prediction_dates,linestyle ='dashed',color = '#ff9999',label = 'Predicted');
plt.plot_date(y=data['y'],x=data['ds'],linestyle = '-',color = 'blue',label = 'Actual');
plt.legend();

> ## Observation
ARIMA model is predicting over 250000 cases in next 15 days in India

<a id="section-eleven"></a>
> ## 4. Lockdown Analysis in India

> ### I have divided the data into 4 parts to analyze the each lockdown phase separately and tried to analyze the affect of Lockdown in India.
> * Prelockdown Period - P0 (Starting from First Case in India till First lockdown on 25th March)
> * Lockdown 1.0 - P1 (25th March to 14th April)
> * Lockdown 2.0 - P2 (Extended till 3rd May)
> * Lockdown 3.0 - P3 (3rd May to Present)


In [ ]:
# Dividing data into 4 phases
tc_phase0 = ts_india[(tc_param['Date']>'2020-03-01') & (ts_india['Date']<'2020-03-25')].reset_index(drop=True) # considering pre lockdown period
tc_phase1 = ts_india[(tc_param['Date']>='2020-03-25') & (ts_india['Date']<='2020-04-14')].reset_index(drop=True) # Lockdown 1.0
tc_phase2 = ts_india[(tc_param['Date']>='2020-04-15') & (ts_india['Date']<='2020-05-03')].reset_index(drop=True) # Lockdown 2.0
tc_phase3 = ts_india[(tc_param['Date']>='2020-05-03') & (ts_india['Date']<'2020-05-17')].reset_index(drop=True) # Lockdown 3.0

tc_cases0 = tc_param[(tc_param['Date']>'2020-03-01') & (ts_india['Date']<'2020-03-25')].reset_index(drop=True) # considering pre lockdown period
tc_cases1 = tc_param[(tc_param['Date']>='2020-03-25') & (ts_india['Date']<='2020-04-14')].reset_index(drop=True) # Lockdown 1.0
tc_cases2 = tc_param[(tc_param['Date']>='2020-04-15') & (ts_india['Date']<='2020-05-03')].reset_index(drop=True) # Lockdown 2.0
tc_cases3 = tc_param[(tc_param['Date']>='2020-05-03') & (ts_india['Date']<'2020-05-17')].reset_index(drop=True) # Lockdown 3.0


tc_test1 = df_ind[(df_ind['Date']>='2020-03-25') & (df_ind['Date']<='2020-04-14')].reset_index(drop=True) # Lockdown 1.0
tc_test2 = df_ind[(df_ind['Date']>='2020-04-15') & (df_ind['Date']<='2020-05-03')].reset_index(drop=True) # Lockdown 2.0
tc_test3 = df_ind[(df_ind['Date']>='2020-05-03') & (df_ind['Date']<'2020-05-17')].reset_index(drop=True) # Lockdown 3.0



> ## We have analyzed the lockdown phases on the basis of-
1. Growth Rate
2. SIR Model
3. Cases over Time
4. Moratlity Rate
5. Recovery Rate


<a id="section-twelve"></a>
> ## 4.1 Growth Factor



>  Growth Factor is calculated as-
>  Where 𝐶 is the number of confirmed cases,
>  Growth Factor=ΔC𝑛/ΔC𝑛−1

<a id="section-thirteen"></a>
> ## 4.2 SIR Model

> We have seen some general behavior of the virus in agregated data, for the country where the coronavirus was originated and for four other interesting countries. There's a lot of information to be extracted from this data; for example, we haven't analyzed the effects of long/lat of countries. However, since our main purpose is to develop a predective model in order to understand the key factors that impact the COVID-19 transmission, I'll move on to one of the most famous epidemiologic models: SIR.

> SIR is a simple model that considers a population that belongs to one of the following states:

> Susceptible (S). The individual hasn't contracted the disease, but she can be infected due to transmisison from infected people
Infected (I). This person has contracted the disease
Recovered/Deceased (R). The disease may lead to one of two destinies: either the person survives, hence developing inmunity to the disease, or the person is deceased.

![Image taken from Lewuathe](http://www.lewuathe.com/assets/img/posts/2020-03-11-covid-19-dynamics-with-sir-model/sir.png)

> There are many versions of this model, considering birth and death (SIRD with demography), with intermediate states, etc. However, since we are in the early stages of the COVID-19 expansion and our interest is focused in the short term, we will consider that people develops immunity (in the long term, immunity may be lost and the COVID-19 may come back within a certain seasonality like the common flu) and there is no transition from recovered to the remaining two states. With this, the differential equations that govern the system are:

> 𝑑𝑆𝑑𝑡=−𝛽𝑆𝐼/𝑁
 
> 𝑑𝐼/𝑑𝑡=𝛽𝑆𝐼/𝑁−𝛾𝐼
 
> 𝑑𝑅/𝑑𝑡=𝛾𝐼
 
> Where  𝛽  is the contagion rate of the pathogen and  𝛾  is the recovery rate.



2.1. Implementing the SIR model
SIR model can be implemented in many ways: from the differential equations governing the system, within a mean field approximation or running the dynamics in a social network (graph). For the sake of simplicity, I'vem chosen the first option, and we will simply run a numerical method (Runge-Kutta) to solve the differential equations system.

The functions governing the dif.eqs. are

In [ ]:
# Susceptible equation
def fa(N, a, b, beta):
    fa = -beta*a*b
    return fa

# Infected equation
def fb(N, a, b, beta, gamma):
    fb = beta*a*b - gamma*b
    return fb

# Recovered/deceased equation
def fc(N, b, gamma):
    fc = gamma*b
    return fc

In order to solve the differential equations system, we develop a 4rth order Runge-Kutta method:

In [ ]:
# Runge-Kutta method of 4rth order for 3 dimensions (susceptible a, infected b and recovered r)
def rK4(N, a, b, c, fa, fb, fc, beta, gamma, hs):
    a1 = fa(N, a, b, beta)*hs
    b1 = fb(N, a, b, beta, gamma)*hs
    c1 = fc(N, b, gamma)*hs
    ak = a + a1*0.5
    bk = b + b1*0.5
    ck = c + c1*0.5
    a2 = fa(N, ak, bk, beta)*hs
    b2 = fb(N, ak, bk, beta, gamma)*hs
    c2 = fc(N, bk, gamma)*hs
    ak = a + a2*0.5
    bk = b + b2*0.5
    ck = c + c2*0.5
    a3 = fa(N, ak, bk, beta)*hs
    b3 = fb(N, ak, bk, beta, gamma)*hs
    c3 = fc(N, bk, gamma)*hs
    ak = a + a3
    bk = b + b3
    ck = c + c3
    a4 = fa(N, ak, bk, beta)*hs
    b4 = fb(N, ak, bk, beta, gamma)*hs
    c4 = fc(N, bk, gamma)*hs
    a = a + (a1 + 2*(a2 + a3) + a4)/6
    b = b + (b1 + 2*(b2 + b3) + b4)/6
    c = c + (c1 + 2*(c2 + c3) + c4)/6
    return a, b, c

And finally, to obtain the evolution of the disease we simply define the initial conditions and call the rk4 method:

In [ ]:
def SIR(N, b0, beta, gamma, hs):
    
    """
    N = total number of population
    beta = transition rate S->I
    gamma = transition rate I->R
    k =  denotes the constant degree distribution of the network (average value for networks in which 
    the probability of finding a node with a different connectivity decays exponentially fast
    hs = jump step of the numerical integration
    """
    
    # Initial condition
    a = float(N-1)/N -b0
    b = float(1)/N +b0
    c = 0.

    sus, inf, rec= [],[],[]
    for i in range(10000): # Run for a certain number of time-steps
        sus.append(a)
        inf.append(b)
        rec.append(c)
        a,b,c = rK4(N, a, b, c, fa, fb, fc, beta, gamma, hs)

    return sus, inf, rec

> ## Fitting SIR Model on Total World Population

Results obtained for N=world population, only one initial infected case,  𝛽=0.7 ,  𝛾=0.2  and a leap pass  ℎ𝑠=0.1  are shown below:

In [ ]:
# Parameters of the model
N = 7800*(10**6)
b0 = 0
beta = 0.7
gamma = 0.2
hs = 0.1

sus, inf, rec = SIR(N, b0, beta, gamma, hs)

f = plt.figure(figsize=(8,5)) 
plt.plot(sus, 'b.', label='susceptible');
plt.plot(inf, 'r.', label='infected');
plt.plot(rec, 'c.', label='recovered/deceased');
plt.title("SIR model")
plt.xlabel("time", fontsize=10);
plt.ylabel("Fraction of population", fontsize=10);
plt.legend(loc='best')
plt.xlim(0,1000)
plt.savefig('SIR_example.png')
plt.show()

> ## Observations:
* The number of infected cases increases for a ceratain time period, and then it eventually decreases given that inviduals recover/dead from the disease
* The opposite happens for the recovered/deceased case

<a id="section-fourteen"></a>
> ## 4.3 Analysing Data Before Lockdown (Phase 0)

In [ ]:
# stacked bar chart

# subset columns
temp = tc_cases0[['Date', 'Total Active Cases', 'Total Deceased', 'Total Recovered']]

# rename columns
temp.columns = ['Date', 'Active', 'Deaths', 'Cured']

# melt data
temp = temp.melt(id_vars="Date", value_vars=['Cured', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')
temp.head()

# plot
fig_2 = px.bar(temp, x="Date", y="Count", color='Case', height=400, 
               title='Cases over time', color_discrete_sequence = [rec, dth, act])
fig_2.show()

In [ ]:
ss = []
for i in tc_phase0.index:
    if(i!= min(tc_phase0.index)):
        lm = tc_phase0.loc[i]['Daily Confirmed']/tc_phase0.loc[i-1]['Daily Confirmed']
    else:
        lm = np.NaN
    ss.append(lm)
        
tc_phase0['Growth_Rate'] = ss


ss1 = []
for i in tc_phase1.index:
    if(i!= min(tc_phase1.index)):
        lm1 = tc_phase1.loc[i]['Daily Confirmed']/tc_phase1.loc[i-1]['Daily Confirmed']
    else:
        lm1 = np.NaN
    ss1.append(lm1)
        
tc_phase1['Growth_Rate'] = ss1


ss2 = []
for i in tc_phase2.index:
    if(i!= min(tc_phase2.index)):
        lm2 = tc_phase2.loc[i]['Daily Confirmed']/tc_phase2.loc[i-1]['Daily Confirmed']
    else:
        lm2 = np.NaN
    ss2.append(lm2)
        
tc_phase2['Growth_Rate'] = ss2

ss3 = []
for i in tc_phase3.index:
    if(i!= min(tc_phase3.index)):
        lm3 = tc_phase3.loc[i]['Daily Confirmed']/tc_phase3.loc[i-1]['Daily Confirmed']
    else:
        lm3 = np.NaN
    ss3.append(lm3)
        
tc_phase3['Growth_Rate'] = ss3




In [ ]:
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=tc_phase0['Date'], y=tc_phase0['Growth_Rate'], name='Growth Factor', \
                         marker=dict(color='#008040')))
fig1.layout.update(title_text='COVID-19 Growth Factor in India w.r.t. Time',xaxis_showgrid=False, yaxis_showgrid=True, width=700, yaxis_title='Growth Factor',
        height=500,font=dict(
#         family="Courier New, monospace",
        size=12,
        color="white"
    ))
print("Mean Growth Rate",tc_phase0['Growth_Rate'].mean())
fig1.layout.plot_bgcolor = '#4d3900'
fig1.layout.paper_bgcolor = '#4d3900'
fig1.show()

In [ ]:
from scipy import integrate, optimize
population = float(1352600000)
country_df = pd.DataFrame()
country_df['Total Confirmed'] = tc_phase0['Total Confirmed'].fillna(0)
country_df = country_df[10:]
country_df['day_count'] = list(range(1,len(country_df)+1))

ydata = [i for i in country_df['Total Confirmed']]
xdata = country_df.day_count
ydata = np.array(ydata, dtype=float)
xdata = np.array(xdata, dtype=float)

N = population
inf0 = ydata[0]
sus0 = N - inf0
rec0 = tc_phase0['Total Recovered'].max()+tc_phase0['Total Deceased'].max()

def sir_model(y, x, beta, gamma):
    sus = -beta * y[0] * y[1] / N
    rec = gamma * y[1]
    inf = -(sus + rec)
    return sus, inf, rec

def fit_odeint(x, beta, gamma):
    return integrate.odeint(sir_model, (sus0, inf0, rec0), x, args=(beta, gamma))[:,1]


popt, pcov = optimize.curve_fit(fit_odeint, xdata, ydata)
fitted = fit_odeint(xdata, *popt)

plt.plot(xdata, ydata, 'o')
plt.plot(xdata, fitted)
plt.title("Fit of SIR model for India infected cases")
plt.ylabel("Population infected")
plt.xlabel("Days")
plt.show()
print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])

In [ ]:
temp = tc_phase0.copy()
temp['Mortality Rate'] = temp['Total Deceased']/temp['Total Confirmed']*100
print("Mean Mortaility Rate",temp['Mortality Rate'].mean())
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['Date'], y=temp['Mortality Rate'],mode='lines+markers',marker_color='red'))
fig.update_layout(title_text = 'Trend of Mortality Rate of India')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [ ]:
temp = tc_phase0.copy()
temp['Recovery Rate'] = temp['Total Recovered']/temp['Total Confirmed']*100
print("Mean Recovery Rate",temp['Total Recovered'].mean())
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['Date'], y=temp['Recovery Rate'],
                    mode='lines+markers',marker_color='green'))
fig.update_layout(title_text = 'Trend of Recovery Rate of India')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

> ### Observation Before Lockdown in India

* > There were over 500 cases when the lockdown was implemented on 25th March.
* >  The mean growth rate is 2.1682900953337017.
* >  Optimal parameters: beta = 1.2126939921477582  and gamma =  1.0530470441778572 from SIR MODEL.
* >  Mean Mortaility Rate 0.9372133853884259.
* >  Mean Recovery Rate 11.608695652173912.







<a id="section-fifteen"></a>
> ## 4.4 Analysis of Lockdown 1.0 (25th March - 14th April)

In [ ]:
# stacked bar chart

# subset columns
temp = tc_cases1[['Date', 'Total Active Cases', 'Total Deceased', 'Total Recovered']]

# rename columns
temp.columns = ['Date', 'Active', 'Deaths', 'Cured']

# melt data
temp = temp.melt(id_vars="Date", value_vars=['Cured', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')
temp.head()

# plot
fig_3 = px.bar(temp, x="Date", y="Count", color='Case', height=540, 
               title='Cases over time', color_discrete_sequence = [rec, dth, act])
fig_3.show()

In [ ]:
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=tc_phase1['Date'], y=tc_phase1['Growth_Rate'], name='Growth Factor', \
                         marker=dict(color='#008040')))
fig2.layout.update(title_text='COVID-19 Growth Factor in India w.r.t. Time',xaxis_showgrid=False, yaxis_showgrid=True, width=700, yaxis_title='Growth Factor',
        height=500,font=dict(
#         family="Courier New, monospace",
        size=12,
        color="white"
    ))
fig2.layout.plot_bgcolor = '#4d3900'
fig2.layout.paper_bgcolor = '#4d3900'
fig2.show()
print("Mean Growth Rate",tc_phase1['Growth_Rate'].mean())

In [ ]:
population = float(1352600000)
country_df1 = pd.DataFrame()
country_df1['Total Confirmed'] = tc_phase1['Total Confirmed'].fillna(0)
country_df1 = country_df1[10:]
country_df1['day_count'] = list(range(1,len(country_df1)+1))

ydata = [i for i in country_df1['Total Confirmed']]
xdata = country_df1.day_count
ydata = np.array(ydata, dtype=float)
xdata = np.array(xdata, dtype=float)

N = population
inf0 = ydata[0]
sus0 = N - inf0
rec0 = rec0 = tc_phase1['Total Recovered'].max()+tc_phase1['Total Deceased'].max()

def sir_model(y, x, beta, gamma):
    sus = -beta * y[0] * y[1] / N
    rec = gamma * y[1]
    inf = -(sus + rec)
    return sus, inf, rec

def fit_odeint(x, beta, gamma):
    return integrate.odeint(sir_model, (sus0, inf0, rec0), x, args=(beta, gamma))[:,1]


popt, pcov = optimize.curve_fit(fit_odeint, xdata, ydata)
fitted = fit_odeint(xdata, *popt)

plt.plot(xdata, ydata, 'o')
plt.plot(xdata, fitted)
plt.title("Fit of SIR model for India infected cases")
plt.ylabel("Population infected")
plt.xlabel("Days")
plt.show()
print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=tc_test1['Date'], y=tc_test1['total_tested'],
                    mode='lines+markers',marker_color='blue'))
fig.update_layout(title_text = 'Rate of Testing')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=tc_test1['Date'], y=tc_test1['daily_tested'],
                    mode='lines+markers',marker_color='blue'))
fig.update_layout(title_text = 'Daily Rate of Testing')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [ ]:
total_tested = tc_test1['total_tested'].max()
total_positive = tc_test1['positive'].max()
#total_tested = tc_test3['total_tested'].max()
#total_positive = tc_test3['positive'].max()
positivecase_ratio = total_positive * 100 / total_tested
pcr = float("{:.2f}".format(positivecase_ratio))
test_million = np.round(1000000*tc_test1['total_tested'].max()/13000000000,2)
print('Total Number of people tested :', total_tested)
print('Total Number of positive cases :',total_positive)
print('Test Conducted per Million People :',test_million)
print('Positive case per Tests [%]:',pcr)
#Overall 
fig = go.Figure(data=[go.Pie(labels=['Total Samples Tested','Positive Cases from tested samples'],
                            values= [total_tested,total_positive],hole =.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value',textfont_size=20,
                  marker=dict(colors=['#263fa3','#cc3c2f'], line=dict(color='#FFFFFF', width=2)))
fig.update_layout(title_text='COVID19 Test Results from ICMR in india',plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
temp = tc_phase1.copy()
temp['Mortality Rate'] = temp['Total Deceased']/temp['Total Confirmed']*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['Date'], y=temp['Mortality Rate'],mode='lines+markers',marker_color='red'))
fig.update_layout(title_text = 'Trend of Mortality Rate of India')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()
print("Mean Mortality Rate",temp['Mortality Rate'].mean())

In [ ]:
temp = tc_phase1.copy()
temp['Recovery Rate'] = temp['Total Recovered']/temp['Total Confirmed']*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['Date'], y=temp['Recovery Rate'],
                    mode='lines+markers',marker_color='green'))
fig.update_layout(title_text = 'Trend of Recovery Rate of India')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()
print("Mean Recovery Rate",temp['Recovery Rate'].mean())

> ## Observation Lockdown 1.0
* >  There were over 10k+ on 14th April out which 1365 have recoverd and 396 > were deceased.
* >  Mean Growth Rate 1.174770643366856
* >  Optimal parameters: beta = 23.599564196488558  and gamma =  > 23.474179051682025 using SIR Model
* >  Total 229.4k testing was done till 14th April.
* >  Daily rate of Testing was in the range of 5k-30k, the testing rate increases from 7th April.

* >  Total Number of people tested : 229426.0
* >  Total Number of positive cases : 10307.0
* >  Test Conducted per Million People : 17.65
* >  Positive case per Tests [%]: 4.49
* >  Mean Mortality Rate[%] 2.831688020930559
* >  Mean Recovery Rate[%] 9.058082827807343

<a id="section-sixteen"></a>
> ## 4.5 Analysis of Lockdown 2.0

In [ ]:
# stacked bar chart

# subset columns
temp = tc_cases2[['Date', 'Total Active Cases', 'Total Deceased', 'Total Recovered']]

# rename columns
temp.columns = ['Date', 'Active', 'Deaths', 'Cured']

# melt data
temp = temp.melt(id_vars="Date", value_vars=['Cured', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')
temp.head()

# plot
fig_4 = px.bar(temp, x="Date", y="Count", color='Case', height=540, 
               title='Cases over time', color_discrete_sequence = [rec, dth, act])
fig_4.show()

In [ ]:
fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=tc_phase2['Date'], y=tc_phase2['Growth_Rate'], name='Growth Factor', \
                         marker=dict(color='#008040')))
fig3.layout.update(title_text='COVID-19 Growth Factor in India w.r.t. Time',xaxis_showgrid=False, yaxis_showgrid=True, width=700, yaxis_title='Growth Factor',
        height=500,font=dict(
#         family="Courier New, monospace",
        size=12,
        color="white"
    ))
fig3.layout.plot_bgcolor = '#4d3900'
fig3.layout.paper_bgcolor = '#4d3900'
fig3.show()
print("Mean Growth Rate",tc_phase2['Growth_Rate'].mean())

In [ ]:
country_df2 = pd.DataFrame()
country_df2['Total Confirmed'] = tc_phase2['Total Confirmed'].fillna(0)
country_df2 = country_df2[10:]
country_df2['day_count'] = list(range(1,len(country_df2)+1))

ydata = [i for i in country_df2['Total Confirmed']]
xdata = country_df2.day_count
ydata = np.array(ydata, dtype=float)
xdata = np.array(xdata, dtype=float)

N = population
inf0 = ydata[0]
sus0 = N - inf0
rec0 = rec0 = tc_phase2['Total Recovered'].max()+tc_phase2['Total Deceased'].max()

def sir_model(y, x, beta, gamma):
    sus = -beta * y[0] * y[1] / N
    rec = gamma * y[1]
    inf = -(sus + rec)
    return sus, inf, rec

def fit_odeint(x, beta, gamma):
    return integrate.odeint(sir_model, (sus0, inf0, rec0), x, args=(beta, gamma))[:,1]


popt, pcov = optimize.curve_fit(fit_odeint, xdata, ydata)
fitted = fit_odeint(xdata, *popt)

plt.plot(xdata, ydata, 'o')
plt.plot(xdata, fitted)
plt.title("Fit of SIR model for India infected cases")
plt.ylabel("Population infected")
plt.xlabel("Days")
plt.show()
print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=tc_test2['Date'], y=tc_test2['total_tested'],
                    mode='lines+markers',marker_color='blue'))
fig.update_layout(title_text = 'Rate of Testing')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=tc_test2['Date'], y=tc_test2['daily_tested'],
                    mode='lines+markers',marker_color='blue'))
fig.update_layout(title_text = 'Daily Rate of Testing')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [ ]:
total_tested = tc_test2['total_tested'].max()
total_positive = tc_test2['positive'].max()
#total_tested = tc_test3['total_tested'].max()
#total_positive = tc_test3['positive'].max()
positivecase_ratio = total_positive * 100 / total_tested
pcr = float("{:.2f}".format(positivecase_ratio))
test_million = np.round(1000000*tc_test2['total_tested'].max()/13000000000,2)
print('Total Number of people tested :', total_tested)
print('Total Number of positive cases :',total_positive)
print('Test Conducted per Million People :',test_million)
print('Positive case per Tests [%]:',pcr)
#Overall 
fig = go.Figure(data=[go.Pie(labels=['Total Samples Tested','Positive Cases from tested samples'],
                            values= [total_tested,total_positive],hole =.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value',textfont_size=20,
                  marker=dict(colors=['#263fa3','#cc3c2f'], line=dict(color='#FFFFFF', width=2)))
fig.update_layout(title_text='COVID19 Test Results from ICMR in india',plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
temp = tc_phase2.copy()
temp['Mortality Rate'] = temp['Total Deceased']/temp['Total Confirmed']*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['Date'], y=temp['Mortality Rate'],mode='lines+markers',marker_color='red'))
fig.update_layout(title_text = 'Trend of Mortality Rate of India')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()
print("Mean Mortality Rate",temp['Mortality Rate'].mean())

In [ ]:
temp = tc_phase2.copy()
temp['Recovery Rate'] = temp['Total Recovered']/temp['Total Confirmed']*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['Date'], y=temp['Recovery Rate'],
                    mode='lines+markers',marker_color='green'))
fig.update_layout(title_text = 'Trend of Recovery Rate of India')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()
print("Mean Recovery Rate",temp['Recovery Rate'].mean())

> ## Obsevation from Lockdown 2.0
* > There were over 29k+ active cases 3rd May out of which 11k have recoverd and 1463 were deceased.
* > Mean Growth Rate 1.0876894606676482
* > Optimal parameters: beta = 0.8143909713954196  and gamma =  0.7550283392636146 using SIR Model
* > About 1M testing was done till 3rd May
* > Daily rate of Testing was in the range of 20k-70k, the testing rate increases from 26th April.
* > Total Number of people tested : 1046450.0
* > Total Number of positive cases : 39980.0
* > Test Conducted per Million People : 80.5
* > Positive case per Tests [%]: 3.82
* > Mean Mortality Rate [%]: 3.261192126339329
* > Mean Recovery Rate[%] 21.144983937598926


<a id="section-seventeen"></a>
> ## 4.6 Analysis of Lockdown 3.0

In [ ]:
# stacked bar chart

# subset columns
temp = tc_cases3[['Date', 'Total Active Cases', 'Total Deceased', 'Total Recovered']]

# rename columns
temp.columns = ['Date', 'Active', 'Deaths', 'Cured']

# melt data
temp = temp.melt(id_vars="Date", value_vars=['Cured', 'Deaths', 'Active'],
                 var_name='Case', value_name='Count')
temp.head()

# plot
fig_5 = px.bar(temp, x="Date", y="Count", color='Case', height=540, 
               title='Cases over time', color_discrete_sequence = [rec, dth, act])
fig_5.show()

In [ ]:
fig4 = go.Figure()
fig4.add_trace(go.Scatter(x=tc_phase3['Date'], y=tc_phase3['Growth_Rate'], name='Growth Factor', \
                         marker=dict(color='#008040')))
fig4.layout.update(title_text='COVID-19 Growth Factor in India w.r.t. Time',xaxis_showgrid=False, yaxis_showgrid=True, width=700, yaxis_title='Growth Factor',
        height=500,font=dict(
#         family="Courier New, monospace",
        size=12,
        color="white"
    ))
fig4.layout.plot_bgcolor = '#4d3900'
fig4.layout.paper_bgcolor = '#4d3900'
fig4.show()
print("Mean Growth Factor",tc_phase3['Growth_Rate'].mean())

In [ ]:
population = float(1352600000)
country_df3 = pd.DataFrame()
country_df3['Total Confirmed'] = tc_phase3['Total Confirmed'].fillna(0)
country_df3 = country_df3[10:]
country_df3['day_count'] = list(range(1,len(country_df3)+1))

y1data = [i for i in country_df3['Total Confirmed']]
x1data = country_df3.day_count
y1data = np.array(ydata, dtype=float)
x1data = np.array(xdata, dtype=float)

N = population
inf0 = ydata[0]
sus0 = N - inf0
rec0 = tc_phase3['Total Recovered'].max()+tc_phase3['Total Deceased'].max()

def sir_model(y, x, beta, gamma):
    sus = -beta * y[0] * y[1] / N
    rec = gamma * y[1]
    inf = -(sus + rec)
    return sus, inf, rec

def fit_odeint(x, beta, gamma):
    return integrate.odeint(sir_model, (sus0, inf0, rec0), x, args=(beta, gamma))[:,1]


popt, pcov = optimize.curve_fit(fit_odeint, x1data, y1data)
fitted = fit_odeint(xdata, *popt)

plt.plot(x1data, y1data, 'o')
plt.plot(x1data, fitted)
plt.title("Fit of SIR model for India infected cases")
plt.ylabel("Population infected")
plt.xlabel("Days")
plt.show()
print("Optimal parameters: beta =", popt[0], " and gamma = ", popt[1])

Where  𝛽  is the contagion rate of the pathogen and  𝛾  is the recovery rate.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=tc_test3['Date'], y=tc_test3['daily_tested'],
                    mode='lines+markers',marker_color='blue'))
fig.update_layout(title_text = 'Daily Testing Rate')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=tc_test3['Date'], y=tc_test3['total_tested'],
                    mode='lines+markers',marker_color='blue'))
fig.update_layout(title_text = 'Testing Rate')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()

In [ ]:
total_tested = tc_test3['total_tested'].max()
total_positive = tc_test3['positive'].max()
#total_tested = tc_test3['total_tested'].max()
#total_positive = tc_test3['positive'].max()
positivecase_ratio = total_positive * 100 / total_tested
pcr = float("{:.2f}".format(positivecase_ratio))
test_million = np.round(1000000*tc_test3['total_tested'].max()/13000000000,2)
print('Total Number of people tested :', total_tested)
print('Total Number of positive cases :',total_positive)
print('Test Conducted per Million People :',test_million)
print('Positive case per Tests [%]:',pcr)
#Overall 
fig = go.Figure(data=[go.Pie(labels=['Total Samples Tested','Positive Cases from tested samples'],
                            values= [total_tested,total_positive],hole =.3)])
fig.update_traces(hoverinfo='label+percent', textinfo='value',textfont_size=20,
                  marker=dict(colors=['#263fa3','#cc3c2f'], line=dict(color='#FFFFFF', width=2)))
fig.update_layout(title_text='COVID19 Test Results from ICMR in india',plot_bgcolor='rgb(275, 270, 273)')
fig.show()

In [ ]:
temp = tc_phase3.copy()
temp['Mortality Rate'] = temp['Total Deceased']/temp['Total Confirmed']*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['Date'], y=temp['Mortality Rate'],mode='lines+markers',marker_color='red'))
fig.update_layout(title_text = 'Trend of Mortality Rate of India')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()
print("Mean Mortality Rate",temp['Mortality Rate'].mean())

In [ ]:
temp = tc_phase3.copy()
temp['Recovery Rate'] = temp['Total Recovered']/temp['Total Confirmed']*100
fig = go.Figure()
fig.add_trace(go.Scatter(x=temp['Date'], y=temp['Recovery Rate'],
                    mode='lines+markers',marker_color='green'))
fig.update_layout(title_text = 'Trend of Recovery Rate of India')
fig.update_layout(plot_bgcolor='rgb(275, 270, 273)',width=600, height=600)
fig.show()
print("Mean Recovery Rate",temp['Recovery Rate'].mean())


> ## Observation of Lockdown 3.0
* >  There were over 53k+ active cases till 17th May out of which 34k have recoverd and 2872 were deceased.
* >  Mean Growth Factor 1.0503204991084447
* >  Optimal parameters: beta = 0.0004055735238853274  and gamma =  -0.05889203836323552 using SIR Model
* >  About 2.2M+ testing was done till 16th May.
* >  Daily rate of Testing was in the range of 60k-95k, the testing rate increases from 13th May.
* >  Total Number of people tested : 2134277.0
* >  Total Number of positive cases : 67152.0
* >  Test Conducted per Million People : 164.18
* >  Positive case per Tests [%]: 3.15  
* >  Mean Mortality Rate[%] 3.3044164774175835
* >  Mean Recovery Rate[%] 31.433540749460498